In [ ]:
import pandas as pd
import numpy as np

def load_and_preprocess_data(train_path, test_path):
    # 데이터 로드
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    # ID 열 제거
    train_df = train_df.set_index('ID')
    test_df = test_df.set_index('ID')
    
    # train 데이터에서 SUBCLASS 열 분리
    train_subclass = train_df['SUBCLASS']
    train_df = train_df.drop('SUBCLASS', axis=1)
    
    # 결측치를 WT로 채우기
    train_df = train_df.fillna('WT')
    test_df = test_df.fillna('WT')
    
    return train_df, test_df, train_subclass

# 데이터 로드 및 전처리
train_df, test_df, train_subclass = load_and_preprocess_data('/kaggle/input/datadata/train.csv', '/kaggle/input/datadata/test.csv')

print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)
print("Train SUBCLASS shape:", train_subclass.shape)

# 처음 5개 행과 5개 열 출력
print("\nTrain data preview:")
print(train_df.iloc[:5, :5])
print("\nTest data preview:")
print(test_df.iloc[:5, :5])
print("\nTrain SUBCLASS preview:")
print(train_subclass.head())

# SUBCLASS의 유니크 값 확인
print("\nUnique SUBCLASS values:")
print(train_subclass.unique())

# 각 SUBCLASS의 개수 확인
print("\nSUBCLASS value counts:")
print(train_subclass.value_counts())

In [ ]:
대회에서 주어진 test , train 데이터를 로드하고 subclass는 타겟변수이기 때문에 정보누출을 막기위해서 따로 분리하여 저장하고 결측치는 wt로 저장하였다

In [ ]:
import numpy as np

# 변이 유형을 순서에 따라 정의 (우선순위가 높은 변이일수록 먼저 처리)
# 1: WT, 2: Multiple Mutations, 3: Synonymous, 4: Nonsynonymous, 5: Nonsense, 6: Frameshift
mutation_types = {
    'WT': 0,
    'Multiple Mutations': 1,
    'Synonymous': 2,
    'Nonsynonymous': 3,
    'Nonsense': 4,
    'Frameshift': 5
}

# 변이 유형을 판별하는 함수들 정의
def is_synonymous(mutation):
    return mutation != 'WT' and len(mutation) > 1 and mutation[0] == mutation[-1]

def is_nonsynonymous(mutation):
    return mutation != 'WT' and not is_synonymous(mutation) and '*' not in mutation and 'fs' not in mutation

def is_nonsense(mutation):
    return '*' in mutation

def is_frameshift(mutation):
    return 'fs' in mutation

def is_multiple_mutations(mutation):
    return ' ' in mutation  # 공백이 있는 경우 여러 변이가 발생한 경우로 처리

def is_wt(mutation):
    return mutation == 'WT'

# 레이블 인코딩 함수 정의 (다중 변이 우선 처리)
def label_encode_mutations(mutation_string):
    # 다중 변이 여부 확인
    if is_multiple_mutations(mutation_string):
        return mutation_types['Multiple Mutations']  # Multiple Mutations 인코딩
    
    # 다중 변이가 아닌 경우, 단일 변이 타입을 우선순위에 따라 처리
    if is_wt(mutation_string):
        return mutation_types['WT']
    elif is_synonymous(mutation_string):
        return mutation_types['Synonymous']
    elif is_nonsynonymous(mutation_string):
        return mutation_types['Nonsynonymous']
    elif is_nonsense(mutation_string):
        return mutation_types['Nonsense']
    elif is_frameshift(mutation_string):
        return mutation_types['Frameshift']

# 각 셀에서 변이 정보를 하나씩 인코딩하여 개별적으로 변환하는 함수
def encode_cell(mutation_string):
    return label_encode_mutations(mutation_string)

# 라벨 인코딩 수행 함수 수정
def label_encode_dataframe(df):
    # 각 열에 대해 인코딩 함수 적용
    encoded_df = df.apply(lambda col: col.map(encode_cell))
    return encoded_df
    

# 라벨 인코딩된 뷰 생성
train_encoded_view = label_encode_dataframe(train_df)
test_encoded_view = label_encode_dataframe(test_df)

# 인코딩된 데이터 확인
print("\nEncoded Train View preview:")
print(train_encoded_view.head())

print("\nEncoded Test View preview:")
print(test_encoded_view.head())



In [ ]:
유전자변이 타입에 따라 라벨 인코딩(view 1)

In [ ]:
import pandas as pd
import numpy as np

def create_binary_mutation_view(df):
    # 변이 유형을 순서에 따라 정의
    mutation_types = ['WT', 'Multiple Mutations', 'Synonymous', 'Nonsynonymous', 'Nonsense', 'Frameshift']

    # 변이 유형을 판별하는 함수들 정의
    def get_mutation_type(mutation):
        if pd.isna(mutation) or mutation == 'WT':
            return 'WT'
        if ' ' in mutation:
            return 'Multiple Mutations'
        if '*' in mutation:
            return 'Nonsense'
        if 'fs' in mutation:
            return 'Frameshift'
        if len(mutation) > 1 and mutation[0] == mutation[-1]:
            return 'Synonymous'
        return 'Nonsynonymous'

    # 변이 유형별 열 생성
    binary_data = {}
    for gene in df.columns:
        for m_type in mutation_types:
            col_name = f"{gene}_{m_type}"
            binary_data[col_name] = []

    # 각 셀을 순회하며 해당하는 열에 1을 설정
    for idx, row in df.iterrows():
        for gene in df.columns:
            mutation_type = get_mutation_type(row[gene])
            for m_type in mutation_types:
                col_name = f"{gene}_{m_type}"
                if mutation_type == m_type:
                    binary_data[col_name].append(1)
                else:
                    binary_data[col_name].append(0)

    # 새로운 이진 변이 데이터프레임 생성
    binary_df = pd.DataFrame(binary_data, index=df.index)

    return binary_df

# ===========================================
# 기존에 정의된 train_df와 test_df 사용
# ===========================================

# 이진 변이 뷰 생성
train_binary_mutation_view = create_binary_mutation_view(train_df)
test_binary_mutation_view = create_binary_mutation_view(test_df)

print("\nTrain Binary Mutation View:")
print(train_binary_mutation_view.head())

print("\nTest Binary Mutation View:")
print(test_binary_mutation_view.head())


In [ ]:
라벨데이터를 원핫인코딩하여 새롭게 저장(view 2)

In [ ]:
import pandas as pd
import numpy as np
import re

def extract_positions(mutation_string):
    """돌연변이 문자열에서 위치 정보를 추출"""
    if pd.isna(mutation_string) or mutation_string == 'WT':
        return []
    return [int(pos) for pos in re.findall(r'[A-Z](\d+)[A-Z*]', mutation_string)]

def find_max_position(df):
    """데이터프레임에서 최대 돌연변이 위치를 찾는 함수"""
    all_positions = df.apply(lambda col: [pos for mutation in col for pos in extract_positions(mutation)])
    max_position = max([max(positions, default=0) for positions in all_positions], default=0)
    return max_position

def create_mutation_statistics_view(df, max_position):
    """돌연변이 통계 정보를 계산하여 뷰를 생성하는 함수"""
    def process_column(column):
        positions_list = column.apply(extract_positions)
        lengths = positions_list.apply(len)

        # Non-empty positions 처리
        non_empty_mask = lengths > 0
        non_empty_positions = positions_list[non_empty_mask]
        
        stats = {
            'count': lengths,
            'mean': np.zeros_like(lengths, dtype=float),
            'std': np.zeros_like(lengths, dtype=float),
            'min': np.zeros_like(lengths, dtype=float),
            'max': np.zeros_like(lengths, dtype=float),
            'range': np.zeros_like(lengths, dtype=float)
        }
        
        if non_empty_positions.any():
            flattened_positions = non_empty_positions.apply(np.array)
            stats['mean'][non_empty_mask] = flattened_positions.apply(np.mean)
            stats['std'][non_empty_mask] = flattened_positions.apply(lambda pos: np.std(pos) if len(pos) > 1 else 0)
            stats['min'][non_empty_mask] = flattened_positions.apply(np.min)
            stats['max'][non_empty_mask] = flattened_positions.apply(np.max)
            stats['range'][non_empty_mask] = stats['max'][non_empty_mask] - stats['min'][non_empty_mask]
        
        return pd.DataFrame(stats)

    # 각 컬럼에 대해 통계 계산 후 합치기
    result = pd.concat({col: process_column(df[col]) for col in df.columns}, axis=1)
    
    # 다중 인덱스를 플랫(flat)하게 변환
    result.columns = [f'{col[0]}_{col[1]}' for col in result.columns]
    
    return result

# 메인 코드
max_position_train = find_max_position(train_df)
max_position_test = find_max_position(test_df)
MAX_POSITION = max(max_position_train, max_position_test)
MAX_POSITION = ((MAX_POSITION // 1000) + 1) * 1000

print(f"Calculated MAX_POSITION: {MAX_POSITION}")

train_position_stats_view = create_mutation_statistics_view(train_df, MAX_POSITION)
test_position_stats_view = create_mutation_statistics_view(test_df, MAX_POSITION)

print("Train Position Statistics view shape:", train_position_stats_view.shape)
print("Test Position Statistics view shape:", test_position_stats_view.shape)


In [ ]:
변이된 유전자의 위치를 표현하기 위해서 position_stats_view를 생성 (view 3) 

In [ ]:
import pandas as pd
import numpy as np

# -----------------------------------
# 1. 데이터 로딩 및 인덱스 설정
# -----------------------------------

census_df = pd.read_csv('/kaggle/input/datadata/Census_allThu.csv')

# -----------------------------------
# 2. 역할 분류 함수 정의
# -----------------------------------

def classify_role(role):
    """
    주어진 역할 문자열을 분류하여 표준화된 역할을 반환합니다.
    """
    if pd.isna(role):
        return 'unknown'
    role_lower = role.lower()
    if 'tsg' in role_lower and 'fusion' in role_lower:
        return 'tsg_fusion'
    if 'oncogene' in role_lower and 'fusion' in role_lower:
        return 'oncogene_fusion'
    if 'tsg' in role_lower:
        return 'tsg'
    if 'oncogene' in role_lower:
        return 'oncogene'
    if 'fusion' in role_lower:
        return 'fusion'
    return 'other'

# -----------------------------------
# 3. 유전자 정보 처리 및 Role_Label 매핑
# -----------------------------------

# 유전자 정보를 'Gene Symbol'을 인덱스로 설정하여 저장
gene_info = census_df.set_index('Gene Symbol')

# 역할 분류 함수 적용
gene_info['Classified_Role'] = gene_info['Role in Cancer'].apply(classify_role)

# 역할별 Role_Label 매핑 정의
# 'unknown'은 0, 'tsg_fusion'은 1, 'oncogene_fusion'은 2, 'tsg'는 3, 'oncogene'은 4, 'fusion'과 'other'는 5로 설정
role_mapping = {
    'unknown': 0,
    'tsg_fusion': 1,
    'oncogene_fusion': 2,
    'tsg': 3,
    'oncogene': 4,
    'fusion': 5,
    'other': 0
}

# 'Role_Label' 열 생성
gene_info['Role_Label'] = gene_info['Classified_Role'].map(role_mapping)

# -----------------------------------
# 4. gene_info 확장: train 및 test의 모든 유전자 포함
# -----------------------------------

# train_df와 test_df의 모든 유전자 목록 결합
all_genes = train_df.columns.union(test_df.columns)

# gene_info를 all_genes에 맞게 재인덱싱, 없는 유전자는 NaN으로 채움
gene_info = gene_info.reindex(all_genes, fill_value=np.nan)

# 'Classified_Role'이 NaN인 유전자는 'unknown'으로 채움
gene_info['Classified_Role'] = gene_info['Classified_Role'].fillna('unknown')

# 'Role_Label'이 NaN인 유전자는 'unknown'의 라벨로 채움 (0)
gene_info['Role_Label'] = gene_info['Role_Label'].fillna(role_mapping['unknown']).astype(int)

# -----------------------------------
# 5. 통합 뷰 생성 함수 정의
# -----------------------------------

def create_integrated_view(df, gene_info):
    """
    주어진 데이터프레임과 gene_info를 사용하여 통합 뷰를 생성합니다.
    유전자가 'WT'가 아닌 경우 해당 유전자의 Role_Label을, 'WT'인 경우 0으로 설정합니다.
    """
    # 데이터프레임 복사
    integrated_view = df.copy()
    
    # 각 유전자에 대해 처리
    for gene in df.columns:
        role_label = gene_info.loc[gene, 'Role_Label']
        # 'WT'가 아닌 경우 role_label, 'WT'인 경우 0으로 설정
        # 'WT'는 문자열로 처리되므로, 데이터 타입이 일치하도록 변환
        integrated_view[gene] = np.where(integrated_view[gene].astype(str) != 'WT', role_label, 0)
    
    return integrated_view

# -----------------------------------
# 6. 통합 뷰 생성
# -----------------------------------

train_integrated_view = create_integrated_view(train_df, gene_info)
test_integrated_view = create_integrated_view(test_df, gene_info)

# -----------------------------------
# 7. 결과 출력 함수 정의
# -----------------------------------

def print_view_sample(view, name):
    """
    통합 뷰의 샘플 데이터를 출력합니다.
    """
    print(f"\n{name} Integrated View:")
    print(f"Shape: {view.shape}")
    print("First 5 rows and up to 5 columns:")
    print(view.iloc[:5, :5])

# -----------------------------------
# 8. 결과 출력
# -----------------------------------

print_view_sample(train_integrated_view, "Train")
print_view_sample(test_integrated_view, "Test")

# -----------------------------------
# 9. Role_Label 매핑 정보 출력
# -----------------------------------

print("\nRole Label Mapping:")
for role, label in role_mapping.items():
    print(f"'{role}': {label}")

# -----------------------------------
# 10. 데이터 형태 출력
# -----------------------------------

print("\nOriginal Data Shapes:")
print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")

print("\nIntegrated View Shapes:")
print(f"Train Integrated View: {train_integrated_view.shape}")
print(f"Test Integrated View: {test_integrated_view.shape}")
# 훈련 데이터에서 0이 아닌 값을 포함하는 열의 수 계산
train_columns_non_zero = (train_integrated_view != 0).any().sum()

# 테스트 데이터에서 0이 아닌 값을 포함하는 열의 수 계산
test_columns_non_zero = (test_integrated_view != 0).any().sum()

print(f"훈련 데이터에서 0이 아닌 값을 포함하는 열의 수: {train_columns_non_zero}")
print(f"테스트 데이터에서 0이 아닌 값을 포함하는 열의 수: {test_columns_non_zero}")import pandas as pd
import numpy as np

# 역할별 Role_Label 매핑
role_mapping = {
    0: 'unknown',
    1: 'tsg_fusion',
    2: 'oncogene_fusion',
    3: 'tsg',
    4: 'oncogene',
    5: 'fusion'
}

def expand_role_columns(df, role_mapping):
    """
    각 유전자에 대해 역할별 열을 생성하고, 해당 역할에 대해 1을 표시하는 이진 표현으로 변환합니다.
    """
    # 모든 열 데이터를 저장할 리스트
    all_columns = []

    # 각 유전자에 대해 역할별로 열 생성
    for gene in df.columns:
        for role_label, role_name in role_mapping.items():
            # 역할별 열 이름 지정 (예: A2M_tsg_fusion)
            column_name = f"{gene}_{role_name}"
            # 해당 역할에 해당하는 값은 1, 나머지는 0으로 설정
            col_data = np.where(df[gene] == role_label, 1, 0)
            all_columns.append(pd.Series(col_data, name=column_name))
    
    # 리스트에 저장된 모든 열을 한 번에 데이터프레임으로 병합
    expanded_df = pd.concat(all_columns, axis=1)
    
    return expanded_df

# 기존 통합 뷰를 사용하여 확장된 역할 열 생성 (train과 test에 대해 적용)
expanded_train_view = expand_role_columns(train_integrated_view, role_mapping)
expanded_test_view = expand_role_columns(test_integrated_view, role_mapping)
import pandas as pd
import numpy as np

# 데이터 로딩
census_df = pd.read_csv('/kaggle/input/datadata/Census_allThu.csv')

# 1단계: census_df에서 유전자 정보를 인덱스로 설정하고 Hallmark 정보를 이진값으로 변환 (Yes -> 1, No -> 0)
gene_info = census_df.set_index('Gene Symbol')
gene_info['Hallmark'] = (gene_info['Hallmark'] == 'Yes').astype(int)

# Hallmark view 생성 함수
def create_hallmark_view(df, gene_info):
    """
    주어진 데이터프레임(df)에서 유전자들이 census_df와 일치하는지 확인하여 
    일치하는 경우 Hallmark 값을 적용하고, 일치하지 않는 유전자는 0으로 설정합니다.
    """
    hallmark_view = df.copy()  # 원본을 복사해서 사용
    for gene in df.columns:
        if gene in gene_info.index:
            # 해당 유전자가 census_df에 존재하면 해당 유전자의 Hallmark 값을 적용
            hallmark_value = gene_info.loc[gene, 'Hallmark']
            hallmark_view[gene] = np.where(hallmark_view[gene] != 'WT', hallmark_value, 0)
        else:
            # 해당 유전자가 census_df에 존재하지 않으면 모든 값을 0으로 설정
            hallmark_view[gene] = 0
    return hallmark_view

# train_df 및 test_df에 대해 Hallmark view 생성
train_hallmark_view = create_hallmark_view(train_df, gene_info)
test_hallmark_view = create_hallmark_view(test_df, gene_info)

# 결과 출력 함수 정의
def print_view_sample(view, name):
    """
    뷰의 첫 5행과 첫 5열을 출력하고 첫 번째 열의 값 카운트를 출력합니다.
    """
    print(f"\n{name} View:")
    print(f"Shape: {view.shape}")
    print("First 5 rows and up to 5 columns:")
    print(view.iloc[:5, :5])
    print("\nValue counts for the first column:")
    print(view.iloc[:, 0].value_counts())

# Train 및 Test 데이터의 Hallmark View 샘플 출력
print_view_sample(train_hallmark_view, "Train Hallmark")
print_view_sample(test_hallmark_view, "Test Hallmark")

# 원본 데이터와 Hallmark view의 shape 출력
print("\nOriginal Data Shapes:")
print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")

print("\nHallmark View Shapes:")
print(f"Train: {train_hallmark_view.shape}")
print(f"Test: {test_hallmark_view.shape}")

# Hallmark 정보 출력
print("\nHallmark Information:")
print(gene_info['Hallmark'].value_counts())
# 훈련 데이터에서 1의 값을 포함하는 열의 수 계산

train_columns_with_ones = (train_hallmark_view == 1).any().sum()

# 테스트 데이터에서 1의 값을 포함하는 열의 수 계산
test_columns_with_ones = (test_hallmark_view == 1).any().sum()

print(f"훈련 데이터에서 1의 값을 포함하는 열의 수: {train_columns_with_ones}")
print(f"테스트 데이터에서 1의 값을 포함하는 열의 수: {test_columns_with_ones}")

In [ ]:
암에 대한 외부데이터를 이용해서 암에 중요하다고 알려진 유전자를 추출하여 유전자가 암의 Hallmark 특징과 관련이 있는지 여부 ,유전자의 암에서의 역할 (예: 종양 유전자, 종양 억제 유전자, 융합 유전자 등).
이에 대해서 3개의 view를 더 정의하였다.

In [ ]:
위에서 여러가지 view를 정의한 이유는 데이터 증강의 목적과 하나의 데이터로는 암을 분류하기 어렵다고 판단하여
데이터를 여러가지 방향으로 살펴보고 여러방향으로 학습시켜 데이터끼리의 미흡점을 보완하기 위해서이다.

In [ ]:
import numpy as np
import xgboost as xgb
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib
# ---------------------------
# 0. 레이블 인코딩
# ---------------------------
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_subclass)

# ---------------------------
# 1. Train 데이터 스케일링 및 희소 행렬로 변환
# ---------------------------

# 이미 정의된 6개의 뷰 (데이터 증강,각 뷰 모두 스케일링 적용)
train_views = [
    train_binary_mutation_view, 
    train_position_stats_view, 
    train_integrated_view,
    expanded_train_view, 
    train_hallmark_view,
    train_encoded_view,
]


# 스케일러 리스트 초기화 및 스케일링
scalers = []
train_sparse_views = []
for i, view in enumerate(train_views):
    scaler = StandardScaler()
    view_scaled = scaler.fit_transform(view)
    joblib.dump(scaler, f'scaler_view_{i}.joblib')  # 각 스케일러 저장
    sparse_view = csr_matrix(view_scaled)
    train_sparse_views.append(sparse_view)
    scalers.append(scaler)

# ---------------------------
# 3. Train/Validation 데이터 분할
# ---------------------------

train_scaled_views_split = []
val_scaled_views_split = []
val_indices_list = []  # 검증 데이터의 인덱스를 저장

# 데이터와 타겟
y_train = y_train_encoded  # 인코딩된 타겟 레이블 사용

# 각 뷰에 대해 train/validation 분리 (Stratified Split 적용)
for view in train_sparse_views:
    X_train, X_val, y_train_split, y_val_split, train_indices, val_indices = train_test_split(
        view, y_train, range(len(view.toarray())), test_size=0.2, random_state=42, stratify=y_train)
    
    train_scaled_views_split.append(X_train)
    val_scaled_views_split.append(X_val)
    val_indices_list.append(val_indices)


In [ ]:
모델에 데이터를 입력하기 전에 각 뷰의 특성을 표준화합니다(StandardScaler 사용). 이는 데이터의 분포를 평균이 0, 표준 편차가 1이 되도록 조정하여, 모델 학습이 특정 특성에 지나치게 치우치지 않도록 합니다.
대부분의 데이터가 0인 6개의 view에 대해서 csr_matrix를 사용하여 스케일링된 데이터를 희소 행렬로 변환했다. (메모리 절약)
이후 동일한 변환을 검증 데이터 또는 테스트 데이터에 적용할 수 있도록, 각 뷰의 스케일러를 joblib을 사용해 파일 저장

In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# 클래스 가중치 계산
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_split), y=y_train_split)
class_weight_dict = dict(zip(np.unique(y_train_split), class_weights))

def calculate_sample_weights(y, class_weight_dict):
    return np.array([class_weight_dict[label] for label in y])

# LightGBM 다중 GPU 사용 설정 및 전체 데이터 학습
models = []
for i, X_train_view in enumerate(train_scaled_views_split):
    model = lgb.LGBMClassifier(device="gpu", boosting_type='gbdt', n_jobs=-1, verbosity=0)
    sample_weights = calculate_sample_weights(y_train_split, class_weight_dict)
    
    # 전체 데이터 학습
    model.fit(X_train_view, y_train_split, sample_weight=sample_weights)
    models.append(model)

# 각 뷰의 검증 데이터로 예측
val_predictions = []
for i, model in enumerate(models):
    pred = model.predict_proba(val_scaled_views_split[i])
    val_predictions.append(pred)

# 스태킹 (신경망 메타 모델)
num_classes = len(np.unique(y_val_split))
y_val_categorical = to_categorical(y_val_split, num_classes=num_classes)

# 메타 모델 학습용 데이터 준비
stacked_predictions = np.hstack([pred for pred in val_predictions])

# 다중 GPU 설정 (TensorFlow)
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    # 신경망 모델 정의
    meta_model_nn = Sequential([
        Input(shape=(stacked_predictions.shape[1],)),  # Input 레이어 추가
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    # 모델 컴파일
    meta_model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 조기 종료 설정
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # 메타 모델 학습 (신경망 모델)
    meta_model_nn.fit(stacked_predictions, y_val_categorical, epochs=50, batch_size=32,
                      validation_split=0.2, callbacks=[early_stopping], verbose=1)

# 검증 데이터의 원래 레이블 복원
y_val_original = train_subclass.iloc[val_indices_list[0]].values

# 검증 데이터에 대한 최종 스태킹 예측
stacked_val_predictions = np.hstack([model.predict_proba(val_scaled_views_split[i]) for i, model in enumerate(models)])
final_stacking_class_predictions_nn = np.argmax(meta_model_nn.predict(stacked_val_predictions), axis=1)

# 성능 평가
final_stacking_class_predictions_original_nn = label_encoder.inverse_transform(final_stacking_class_predictions_nn)

# 결과 출력
print("\n==== 스태킹 결과 (신경망 모델) ====")
print(f"Validation Accuracy (Stacking with NN): {accuracy_score(y_val_original, final_stacking_class_predictions_original_nn)}")
print("\nClassification Report (Stacking with NN):")
print(classification_report(y_val_original, final_stacking_class_predictions_original_nn))


In [ ]:
클래스의 비율이 불균형하여 class weight를 적용하여 가중치를 주었다.
여러가지 머신러닝 모델을 비교해보고 성능이 가장 좋았던 lightgbm을 이용하여 view들을 각각 학습하였고 최종 신경망 모델을 이용하여 스테킹하여 결과를 종합적으로 예측하였다.

In [ ]:
==== 스태킹 결과 (신경망 모델) ====
Validation Accuracy (Stacking with NN): 0.5318291700241741

Classification Report (Stacking with NN):
              precision    recall  f1-score   support

         ACC       0.92      0.86      0.89        14
        BLCA       0.67      0.19      0.30        21
        BRCA       0.42      0.69      0.52       157
        CESC       0.50      0.13      0.21        31
        COAD       0.74      0.64      0.69        45
        DLBC       1.00      0.43      0.60         7
      GBMLGG       0.63      0.67      0.65        92
        HNSC       0.51      0.47      0.49        45
       KIPAN       0.59      0.70      0.64       103
        KIRC       0.72      0.79      0.75        67
        LAML       0.58      0.47      0.52        32
         LGG       0.72      0.67      0.70        46
        LIHC       0.61      0.35      0.45        31
        LUAD       0.33      0.19      0.24        37
        LUSC       0.75      0.33      0.46        36
          OV       0.22      0.33      0.26        51
        PAAD       0.00      0.00      0.00        24
        PCPG       0.34      0.38      0.36        29
        PRAD       0.33      0.26      0.29        53
        SARC       0.38      0.12      0.19        40
        SKCM       0.76      0.71      0.74        55
        STES       0.43      0.54      0.48        76
        TGCT       0.67      0.40      0.50        25
        THCA       0.59      0.78      0.68        65
        THYM       0.00      0.00      0.00        19
        UCEC       0.74      0.70      0.72        40

    accuracy                           0.53      1241
   macro avg       0.54      0.45      0.47      1241
weighted avg       0.53      0.53      0.51      1241

In [ ]:
# ---------------------------
# 6. 테스트 데이터 예측
# ---------------------------
print("===== 테스트 데이터 예측 시작 =====")

test_views = [
    test_binary_mutation_view, 
    test_position_stats_view, 
    test_integrated_view,
    expanded_test_view, 
    test_hallmark_view,
    test_encoded_view
]

# 테스트 데이터 스케일링 및 희소 행렬 변환
test_scaled_views = []
print("\n===== 테스트 데이터 스케일링 시작 =====")
for i, view in enumerate(test_views):
    scaler_path = f'scaler_view_{i}.joblib'
    print(f"\n[View {i}] 스케일러 파일 로드 시도: {scaler_path}")
    try:
        scaler = joblib.load(scaler_path)  # 학습 시 저장한 스케일러 로드
        print(f"[View {i}] 스케일러 로드 성공")
    except FileNotFoundError:
        raise FileNotFoundError(f"스케일러 파일 {scaler_path}을(를) 찾을 수 없습니다.")
    
    view_scaled = scaler.transform(view)  # 스케일링 (transform만 수행)
    print(f"[View {i}] 스케일링 완료. 스케일링된 데이터 형태: {view_scaled.shape}")
    
    sparse_view = csr_matrix(view_scaled)  # 희소 행렬로 변환
    test_scaled_views.append(sparse_view)
    print(f"[View {i}] 희소 행렬로 변환 완료. 형식: {type(sparse_view)}, 형태: {sparse_view.shape}")
print("===== 테스트 데이터 스케일링 완료 =====\n")

# 기본 모델을 사용한 테스트 데이터 예측 확률 생성
test_predictions = []
print("===== 테스트 데이터 예측 확률 생성 시작 =====")
for i, model in enumerate(models):
    print(f"\n[Model {i}] 예측 확률 생성 중...")
    pred_proba = model.predict_proba(test_scaled_views[i])
    test_predictions.append(pred_proba)
    print(f"[Model {i}] 예측 확률 생성 완료. 예측 확률 형태: {pred_proba.shape}")
print("===== 테스트 데이터 예측 확률 생성 완료 =====\n")

# 예측 확률 스태킹
print("===== 예측 확률 스태킹 시작 =====")
stacked_test_predictions = np.hstack(test_predictions)
print(f"스태킹된 예측 확률의 형태: {stacked_test_predictions.shape}")
print("===== 예측 확률 스태킹 완료 =====\n")

# 메타 모델을 사용한 최종 예측
print("===== 메타 모델을 사용한 최종 예측 시작 =====")
final_test_predictions = meta_model_lgb.predict(stacked_test_predictions)
print(f"최종 예측 결과의 형태: {final_test_predictions.shape}")

# 레이블 인코딩 역변환
print("===== 레이블 인코딩 역변환 시작 =====")
try:
    final_test_labels = label_encoder.inverse_transform(final_test_predictions)
    print("레이블 인코딩 역변환 완료")
except ValueError as e:
    print("레이블 인코딩 역변환 중 오류 발생:", e)
    raise

# 테스트 데이터에 'subclass_predict' 열 생성 및 예측값 할당
print("===== 테스트 데이터에 예측된 'subclass_predict' 열 할당 시작 =====")
if 'subclass_predict' in test_df.columns:
    print("'subclass_predict' 열이 이미 존재합니다. 기존 열을 덮어씁니다.")
    test_df['subclass_predict'] = final_test_labels
else:
    test_df = pd.concat([
        test_df.reset_index(drop=True),  # 인덱스 일치 보장
        pd.Series(final_test_labels, name='subclass_predict')
    ], axis=1)
    print(f"'subclass_predict' 열이 테스트 데이터에 성공적으로 추가되었습니다. 테스트 데이터 형태: {test_df.shape}")
print("===== 테스트 데이터에 예측된 'subclass_predict' 열 할당 완료 =====\n")

# 결과 미리보기
print("===== 예측된 'subclass_predict'가 포함된 테스트 데이터 미리보기 =====")
print(test_df[['subclass_predict']].head())
print("===== 테스트 데이터 예측 미리보기 완료 =====\n")

# 예측 결과를 CSV 파일로 저장
output_csv = 'test_with_predictions.csv'
test_df.to_csv(output_csv, index=False)
print(f"===== 예측 결과가 '{output_csv}'에 저장되었습니다 =====\n")

In [ ]:
이후 검증때와 동일한 방법으로 test데이터를 예측하고 결과를 csv파일로 저장하였다.